In [76]:
import torch
from torch import nn
import einops

In [40]:
with open('/Users/Darrell/Desktop/tiny-shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [41]:
vocab = sorted(list(set(text)))
vocab_size = len(vocab)

In [45]:
stoi = {s:i for i, s in enumerate(vocab)}
itos = {i:s for i, s in enumerate(vocab)}
tokenize = lambda s: [stoi[c] for c in s]
detokenize = lambda c: ''.join([itos[x] for x in c])

In [50]:
class Token_Embedding(nn.Module):
    def __init__(self, vocab_size, emb_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim)

    def forward(self, x):
        return self.embedding(x)


In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, n_head, n_embd, attention_drop = 0.1, residual_drop = 0.1):
        super().__init__()
        self.n_head = n_head
        self.attention_drop = attention_drop
        self.residual_drop = residual_drop
        self.query = nn.Linear(n_embd, n_embd)
        self.key = nn.Linear(n_embd, n_embd)
    
    def forward(self, x):
        # x = (B, T, E) ---> (B, num_heads, T, h_size)
        B, T, E = x.shape
        q = self.query(x).view(B, T, n_head, h_size).transpose(1, 2)
        k = self.key(x).view(B, T, n_head, h_size).transpose(1, 2).transpose(-1, -2)
        W = q @ k * E ** -0.5

        


In [77]:
#Things to implement:
#   1. Flash Attention
#   2. Multi-Query Attention
#   3. Rotary Positional Embedding
#   4. GLU activations

In [79]:
torch.arange(0, 12, 2) + 0.4

tensor([ 0.4000,  2.4000,  4.4000,  6.4000,  8.4000, 10.4000])

In [80]:
x = torch.zeros(2, 3)